# MS 141 Lecture 4 (Part 2)
# SciPy

In [ ]:
# embed plots within the notebook
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

The SciPy library can be employed for a wide range of problems, including:

* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Differential equations ([spipy.integrate.ode](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.ode.html))
* Linear algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))

We will discuss examples of how to use a few of these subpackages.<br> 
We begin by importing everything from the `scipy` module.

In [ ]:
from scipy import *

If we only need to use part of SciPy, we can import only the modules we are interested in.<br> 
For example, we can import only the linear algebra package, under the name `la`:

In [ ]:
import scipy.linalg as la

## Numerical Integration

Numerical evaluation of an integral over one variable:

$\displaystyle \int_a^b f(x)\, dx$

This numerical integration is called *numerical quadrature* (or simply *quadrature*).<br>
SciPy can carry out quadrature using `quad`, `dblquad` and `tplquad` for single, double and triple integrals, respectively.

In [ ]:
from scipy.integrate import quad, dblquad, tplquad

Here is a simple example

In [ ]:
# define a simple function for the integrand
def f(x):
    return x**2

a = 0 # lower limit of x
b = 1 # upper limit of x

val, abserr = quad(f, a, b) # a tuple with two entries, (integral value, error)

print ("integral value =", val, ", absolute error =", abserr)

A more compact form can be achieved with the `lambda` function definition

In [ ]:
# one-line integral with python
val, abserr = quad(lambda x: x ** 2, 0,1)
print (val, abserr)

We can also integrate out to infinity. Here is an example for the well-known Gaussian integral:

In [ ]:
val, abserr = quad(lambda x: exp(-x ** 2), -Inf, Inf)
print ("numerical result =", val, abserr)
print ("analytic result = ", sqrt(pi))

Higher-dimensional integration works in a similar way:

In [ ]:
def integrand(x, y):
    return exp(-x**2-y**2)

x_lower = -Inf  
x_upper = Inf
y_lower = -Inf
y_upper = Inf

val, abserr = dblquad(integrand, x_lower, x_upper, y_lower, y_upper)
print (val, abserr)

## Solving Ordinary Differential Equations (ODEs)

Later in the course, we will discuss approaches to solve ODEs, using which we can write our own ODE solvers.<br> 
SciPy however provides two versatile tools for solving ODEs, one is the function `odeint`, 
and the other is the class `ode`.<br> 
Typically `odeint` is easier to use, but `ode` offers better control.<br> 
A more recent option is the package `solve_ivp`, which adds new functionalities for initial value problems.<br>

Let us import `odeint` from the `scipy.integrate` module. 

In [ ]:
from scipy.integrate import odeint

We can solve a simple first-order ODE:
$$ \frac{dy}{dx} = -y, \hspace{20pt} y(0)=1$$
As you can verify, this has a closed form solution
$$ y (x)= e^{-x}$$
Let's solve it numerically using `odeint` (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html)).

In [ ]:
# Define a function that calculates the derivative
def dy_dx(y, x):
    return -y

xs = np.linspace(0,5,100)
y0 = 1.0  # the initial condition

ys = odeint(dy_dx, y0, xs) 
# optional: print it out (ys is a 2D numpy array)
# print(ys[:,0])

# for comparison, generate the analytic solution at a few points
xexact = np.linspace(0,5,20)
yexact = np.exp(-xexact)

In [ ]:
# Plot and compare the numerical and analytic solutions
plt.xlabel("x")
plt.ylabel("y")
plt.plot(xs, ys,lw=2, label='numerical')
plt.plot (xexact,yexact,'ro', label='exact')
plt.legend()
plt.show();

We'll see more examples of ODEs later in the course.

## Linear algebra

The SciPy [linear algebra](http://docs.scipy.org/doc/scipy/reference/linalg.html) module contains matrix related functions. It can solve linear systems of equations and eigenvalue problems<br> using a range of decompositions (LU, Cholesky, SVD) and methods implemented in LAPACK.<br> 
It can also compute matrix functions such as the trace, determinant, characteristic equation, etc.<br>
Let's import the module and discuss a few examples.

In [ ]:
import scipy.linalg as la

### Matrix operations

In [ ]:
M = np.array([[1,2],[0,1]])
print (M)

In [ ]:
# compute the determinant of M
la.det(M)

In [ ]:
# compute the inverse of M
Minv = la.inv(M)
print (Minv)

In [ ]:
# check the inverse
print (M@Minv)

In [ ]:
# compute the matrix exponential e^M = I + M + M^2/2 + M^3/3! + ...
Mx = np.array([[0,1],[1,0]]) #the square of this matrix is the identity
print (la.expm(Mx))

In [ ]:
# check the result
# since M**2 = I, we have that 
# e^M = (I + I/2 + I/4! ...) + M(I + I/3! + I/5! + ...) = I cosh(1) + M sinh(1)
Mcheck = np.eye(2)*np.cosh(1) + Mx*np.sinh(1)
print (Mcheck - la.expm(Mx))

In [ ]:
# compute the LU decomposition (with partial pivoting) of M
# this decomposes M into the matrices P, L, U, 
# where L is a lower triangular and U an upper triangular matrix, and P is a permutation matrix. 
P, L, U = la.lu(M)
print (P)
print (L)
print (U) # U = M in this example

In [ ]:
# check the LU decomposition
(P @ L @ U) - M

### Linear systems

Solve a linear system of equations of the form

$A\, x = b$

where $A$ is a matrix and $x$ and $b$ are vectors.<br> 
For a square matrix A, it can be solved using the `scipy.linalg` package with the `solve` method, which is a wrapper to the respective LAPACK routines.

In [ ]:
import scipy.linalg as la

In [ ]:
A = np.array([[1,2,2], [0,1,0], [2,0,0]])
b = np.array([1,2,3])
print (A, '\n\n', b)

In [ ]:
x = la.solve(A, b)
print (x)

In [ ]:
# check the solution
print (np.dot(A, x) - b)

### Eigenvalues and eigenvectors

Solve the eigenvalue problem for a matrix $A$:

$\displaystyle A v_n = \lambda_n v_n$

where $v_n$ is the $n$th eigenvector and $\lambda_n$ the corresponding eigenvalue.

The function `eigvals` computes only the eigenvalues, while `eig` computes both the eigenvalues and eigenvectors:

In [ ]:
# example
A = np.array([[0,1],[1,0]]) # note that A^2 = I
evals = la.eigvals(A) # the eigenvalues are +1 and -1, since A^2 = I
print (evals)

In [ ]:
evals, evecs = la.eig(A)
print (evals) 
print ('')
print (evecs) # the eigenvectors are (1,-1) and (1,1) (apart from a normalization factor 1/sqrt(2) )

Note that the $n$th eigenvalue is stored in `evals[n]` and the corresponding eigenvector is the $n$th column in `evecs`, namely `evecs[:,n]`. Let's verify:

In [ ]:
for n in range(np.ndim(A)):
    # compute the vector A v_n - lambda_n v_n
    check = la.norm( np.dot(A, evecs[:,n]) - evals[n] * evecs[:,n])
    print (check)

## Interpolation

Interpolation is the process of finding a value between two points in a data set. Ideally, one can obtain an analytic function that interpolates the data.

In SciPy, the `interp1d` function acts on a data set (x,y), <br>
and returns a function that can give the interpolated value of y for arbitrary x in the range covered by the data.

In [ ]:
from scipy.interpolate import interp1d #interp1d is in scipy.interpolate
import matplotlib.pyplot as plt

# generate data (according to a function)
x = np.linspace(0, 5, 11)
f = lambda x: np.cos(x**2/3+4) 

y = f(x)

# plot the data
plt.plot(x, y,'o')
plt.show()

Create interpolation functions with two different methods

In [ ]:
f1 = interp1d(x, y, kind = 'linear') # linear fit
f2 = interp1d(x, y, kind = 'cubic') # cubit fit

In [ ]:
xnew = np.linspace(0, 5,30)

plt.figure(figsize=(12, 6))
plt.plot(x, y, 'ro', xnew, f1(xnew), 'm-', xnew, f2(xnew), 'k--', xnew,f(xnew), '-')
plt.legend(['data', 'linear', 'cubic','exact'], loc = 'best')
plt.show()

A general interpolation method is the one based on **splines**. See the [documentation](https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html) for more details.

## Statistics

The `scipy.stats` module can be used for statistical distributions, functions and tests.<br> 
Its documentation can be found at http://docs.scipy.org/doc/scipy/reference/stats.html.

In [ ]:
from scipy import stats

In [ ]:
# create a continuous random variable Y with normal distribution
# in this standard definition, the distribution is centered at 0 and has $\sigma = 1$
Y = stats.norm()

In [ ]:
x = linspace(-4,4,100)

fig, axes = plt.subplots(2,1, sharex=True)

# plot the probability distribution function (PDF)
axes[0].plot(x, Y.pdf(x), color='g')

# plot a histogram of 10000 random realizations of the stochastic variable Y
axes[1].hist(Y.rvs(size=10000), bins=50, color='g');

We can compute the mean, standard deviation, and variance of the normal distribution:

In [ ]:
Y.mean(), Y.std(), Y.var() # normal distribution

We can also compute the mean, standard deviation, and variance of any other data set.<br>
Let's try with a data set made up by points distributed according to the normal distribution.<br> 
In the limit of a large number of points, the mean, standard deviation and variance will approach those of the normal distribution

In [ ]:
X1 = Y.rvs(size=10000)
X2 = Y.rvs(size=100)
#print (type(X), len(X))

In [ ]:
print ('X1 (10000 pts):',X1.mean(), X1.std(), X1.var())
print ('')
print ('X2 (100 pts):', X2.mean(), X2.std(), X2.var())

Another python package for statistical modelling worth checking out is [statsmodels](http://statsmodels.sourceforge.net).